# Lab 01 Assignments <br>

#### Name & Student ID
#### 1. Muhammad 'Umar bin Zolkifle (SW01082397)
#### 2. Muhammad Iman Arif bin Mauzi (SW01083215)

##### Using the Reddit as the platform. Below is the link to the website that will be scraped.
##### https://www.reddit.com/r/samsung/comments/1ike0dp/my_thoughts_on_galaxy_s25_ultra_after_using_it/ <br>

#### I'll be using selenium as the webdriver to view and collect the comments from the subreddit.
#### At first, I proceed by installing Selenium and import pandas to collect and view the data.

In [9]:
pip install selenium pandas

Note: you may need to restart the kernel to use updated packages.


##### There are a few functions under in the python file.
#### 1. click_load_more()
##### This function acts to click "View more comments" buttone when the browser scroll down. <br>

#### 2. click_continue_thread()
##### This function acts to click on "Continue this thread" when it appears to load more comments <br>

##### As for the rest of the codes, I utilised the loop to load and scroll down the webpages to view more comments and review of the product.
##### To specifically collect the usernames, time, and review. I inspect the elements of the webpages, identify the specific tag of the elements and put in the python to locate the elements.
##### As of now, the username is defined by its <a tag with css, the time is by TIME and the review is through <p tag. <br>

##### Then, when the webdriver reach the end of the pages. It then closes and export the collected data and commments into csv files to be further processsed. <br>

### Note: I did not do any preprocessing on the data as the comments are complete

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.edge.service import Service
import pandas as pd
import time

# Set up Edge WebDriver
edge_driver_path = "C:\\DataC\\edgedriver_win64\\msedgedriver.exe"  # Update this path
service = Service(edge_driver_path)
driver = webdriver.Edge(service=service)

# Open the Reddit post
url = "https://www.reddit.com/r/samsung/comments/1ike0dp/my_thoughts_on_galaxy_s25_ultra_after_using_it/"
driver.get(url)

# Wait for the page to load
time.sleep(5)

# Function to click "Load more comments" buttons
def click_load_more():
    while True:
        try:
            load_more_buttons = driver.find_elements(By.XPATH, "//button[contains(text(),'View more comments')]")
            if not load_more_buttons:
                break  # No more buttons to click
            
            for button in load_more_buttons:
                ActionChains(driver).move_to_element(button).click(button).perform()
                time.sleep(2)  # Wait for new comments to load
        except:
            break  # If button disappears or fails to click

# Function to click "Continue this thread" links
def click_continue_thread():
    while True:
        try:
            continue_buttons = driver.find_elements(By.XPATH, "//a[contains(text(),'Continue this thread')]")
            if not continue_buttons:
                break  # No more threads to load
            
            for button in continue_buttons:
                driver.execute_script("arguments[0].scrollIntoView();", button)
                button.click()
                time.sleep(2)  # Wait for new comments to load
        except:
            break  # If button disappears or fails to click

# Load more comments dynamically
for _ in range(10):  # Adjust loop count for more comments
    click_load_more()
    click_continue_thread()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

# Store extracted data
reviews_data = []

# Extract comments
comments = driver.find_elements(By.CSS_SELECTOR, "shreddit-comment")

for comment in comments:
    try:
        # Extract user name using the updated <a> tag
        user_name = comment.find_element(By.CSS_SELECTOR, "a.truncate.font-bold.text-neutral-content-strong").text

        # Extract review date
        review_date = comment.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

        # Extract user review
        review_text = comment.find_element(By.CSS_SELECTOR, "p").text

        # Append data to list
        reviews_data.append({"User Name": user_name, "Review Date": review_date, "User Review": review_text})

    except:
        continue  # Skip if any element is missing

# Close the browser
driver.quit()

# Save to CSV
df = pd.DataFrame(reviews_data)
df.to_csv("reddit_comments.csv", index=False)

print("Scraping complete! Comments saved in 'reddit_comments.csv'.")

Scraping complete! Comments saved in 'reddit_comments.csv'.
